In [ ]:
# setup
import numpy as np
import pandas as pd

from scipy.stats import norm, expon, sem

from utility import *
from theoretical import *

pd.set_option("display.float_format", "{:.4f}".format)

# Chapter 4 - Monte Carlo Simulation
## Examples

**Example 4.1. Simulate $W_T$.** Consider the problem of estimating the price
of a call option under the assumption that the underlying stock price is a
geometric Brownian motion.

In [ ]:
# example 4.1
def example_4_1():
    rng = np.random.default_rng(4198)
    
    # simulation params
    s_0, r, sig, T = 50, 0.05, 0.2, 1
    
    def plain(n, k):
        # sampling
        z = norm.rvs(0, 1, size=n, random_state=rng)
        y = s_0 * np.exp((r - 0.5 * sig**2) * T + sig * np.sqrt(T) * z)
        x = np.exp(-r * T) * np.maximum(y - k, 0)

        # compute mean est. and std. error
        v_hat = np.mean(x)
        se = sem(x)

        return v_hat, se

    theoretical = {}
    mc_est = {}
    se_vals = {}
    for n in [2_500, 10_000]:
        for k in [40, 50, 60]:
            theoretical_val = european_call(s_0, k, r, T, sig) 
            mean_est, std_err = plain(n, k)

            key = (f"n={n}", f"{k}")
            theoretical[key] = theoretical_val
            mc_est[key] = mean_est
            se_vals[key] = std_err
    
    df = pd.DataFrame(
        [theoretical, mc_est, se_vals],
        index=["Theoretical value", "M.C. Estimate", "S.E."]
    )

    # Ensure MultiIndex columns
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=["", "Strike price K"])

    return df

In [3]:
results_4_1 = example_4_1()
display(results_4_1)


n=2500               n=10000              
Strike price K         40     50     60      40     50     60
Theoretical value 12.2944 5.2253 1.6237 12.2944 5.2253 1.6237
M.C. Estimate     12.1354 5.1197 1.6504 12.3627 5.2388 1.6388
S.E.               0.1896 0.1451 0.0886  0.0963 0.0727 0.0436

**Example 4.2. Simulate a Brownian Motion Sample Path.** Consider a dis-
cretely monitored average price call option whose payoff at maturity $T$ is
$$\left( \frac{1}{m} \sum_{i=1}^{m} S_{t_i} - K\right)^+,$$
where $0 < t_1 < \cdots < t_m = T$ are a fixed set of dates. Assume that under
the risk-neutral probability measure,
$$S_t = S_0 \exp \left\{ \left( r - \frac{1}{2} \sigma^2 \right) t + \sigma W_t \right\}.$$
Estimate the price of the option.

In [ ]:
# example 4.2
def example_4_2():
    rng = np.random.default_rng(4198)
    
    # simulation params
    s_0, r, sig, T = 50, 0.05, 0.2, 1
    m = 12
    t = [(i+1)/m for i in range(m)]

    def plain(n, k):
        z = norm.rvs(0, 1, size=(m, n), random_state=rng)
        st = np.zeros(shape=(m, n))

        t_diff = [t[0]] + [t[i] - t[i-1] for i in range(1, m)]
        
        st[0] = s_0 * np.exp((r - 1/2 * sig**2) * t_diff[0] + sig * np.sqrt(t_diff[0]) * z[0])
        for i in range(1, m):
            st[i] = st[i-1] * np.exp((r - 1/2 * sig**2) * t_diff[i] + sig * np.sqrt(t_diff[i]) * z[i])
        
        x = np.exp(-r*T) * np.maximum(np.mean(st, axis=0) - k, 0)

        # compute mean est. and std. error
        v_hat = np.mean(x)
        se = sem(x)

        return v_hat, se


    mc_est = {}
    se_vals = {}
    for n in [2_500, 10_000]:
        for k in [40, 50, 60]: 
            mean_est, std_err = plain(n, k)

            key = (f"Sample size n={n}", f"{k}")
            mc_est[key] = mean_est
            se_vals[key] = std_err
    
    df = pd.DataFrame(
        [mc_est, se_vals],
        index=["M.C. Estimate", "S.E."]
    )

    # Ensure MultiIndex columns
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=["", "Strike price K"])

    return df
    

In [5]:
results_4_2 = example_4_2()
results_4_2

Sample size n=2500               Sample size n=10000         \
Strike price K                 40     50     60                  40     50   
M.C. Estimate             10.8402 3.0368 0.2759             10.8829 3.0605   
S.E.                       0.1195 0.0824 0.0254              0.0592 0.0433   

                       
Strike price K     60  
M.C. Estimate  0.3284  
S.E.           0.0140

**Example 4.3. Simulate 2D Jointly Normal Random Vectors.** Estimate the
price of a spread call option whose payoff at maturity $T$ is
$$(X_T - Y_T - K)^+,$$
where $\{X_t\}$ and $\{Y_t\}$ are the prices of two underlying assets. Assume that
under the risk-neutral probability measure,
$$\begin{align*}
X_t &= X_0 \exp \left\{ \left(r - \frac{1}{2} \sigma_1^2 \right) t + \sigma_1 W_t \right\}, \\
Y_t &= X_0 \exp \left\{ \left(r - \frac{1}{2} \sigma_2^2 \right) t + \sigma_2 B_t \right\},
\end{align*}$$
where $(W, B)$ is a two-dimensional Brownian motion with covariance matrix
$$\Sigma = \begin{bmatrix}
    1 & \rho \\
    \rho & 1
\end{bmatrix}.$$

In [ ]:
# example 4.3
def example_4_3():
    rng = np.random.default_rng(4198)
    
    # simulation params
    X_0, Y_0, r = 50, 45, 0.05
    sig1, sig2, rho = 0.2, 0.3, 0.5
    T = 1

    C = np.array([
        [1, 0],
        [rho, np.sqrt(1 - rho**2)]
    ])
    
    def plain(n, k):
        Z = norm.rvs(0, 1, size=(2, n), random_state=rng)
        R = C @ Z
        X = X_0 * np.exp((r - 1/2 * sig1**2) * T + sig1 * np.sqrt(T) * R[0,])
        Y = Y_0 * np.exp((r - 1/2 * sig2**2) * T + sig2 * np.sqrt(T) * R[1,])
        H = np.exp(-r * T) * np.maximum(X - Y - k, 0)

        # compute mean est. and std. error
        v_hat = np.mean(H)
        se = sem(H)

        return v_hat, se


    mc_est = {}
    se_vals = {}
    for n in [2_500, 10_000]:
        for k in [0, 5, 10]: 
            mean_est, std_err = plain(n, k)

            key = (f"Sample size n={n}", f"{k}")
            mc_est[key] = mean_est
            se_vals[key] = std_err
    
    df = pd.DataFrame(
        [mc_est, se_vals],
        index=["M.C. Estimate", "S.E."]
    )

    # Ensure MultiIndex columns
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=["", "Strike price K"])

    return df
    

In [7]:
results_4_3 = example_4_3()
results_4_3

Sample size n=2500               Sample size n=10000         \
Strike price K                  0      5     10                   0      5   
M.C. Estimate              7.9421 4.8720 2.8732              8.0081 4.8200   
S.E.                       0.1680 0.1325 0.1038              0.0836 0.0663   

                       
Strike price K     10  
M.C. Estimate  2.7630  
S.E.           0.0516

**Example 4.4. Simulate Value-at-Risk.** Denote by $X_i$ the daily return of a
portfolio. Assume that $X = \{X_1, X_2, ...\}$ is a Markov chain, that is, the
conditional distribution of $X_{i+1}$ given $(X_i, X_{i−1}, ..., X_1)$ only
depends on $X_i$ for each $i$. Let $X$ be autoregressive conditional
heteroskedastic (ARCH) in the sense that given $X_i = x$, $X_{i+1}$ is normally
distributed with mean zero and variance $\beta_0 + \beta_1 x^2$ for some $\beta_0 > 0$ and $0 < \beta_1 < 1$. The total return within an $m$-day period is
$$S = \sum_{i = 1}^{m} X_i$$
Assuming that $X_1$ is a standard normal random variable, estimate the
value-at-risk at the confidence level $1 − p$.

In [8]:
# example 4.4
def example_4_4():
    rng = np.random.default_rng(4198)
    
    # simulation params
    beta_0, beta_1 = 0.5, 0.5
    alpha = 0.05
    m = 10
    n = 10_000
    
    def plain(n, p):
        
        X = np.zeros(shape=(m, n))
        X[0,:] = norm.rvs(0, 1, size=n, random_state=rng)
        for i in range(1,m):
            Z = norm.rvs(0, 1, size=n, random_state=rng)
            X[i,:] = np.sqrt(beta_0 + beta_1 * X[i-1,:]**2) * Z
        
        S = np.sum(X, axis=0)
        k = int(n * p)

        S_ord = np.sort(S)
        x_hat =  - S_ord[k]

        k1 = int(n * p - np.sqrt(n * p * (1 - p)) * norm.ppf(q=alpha/2))
        k2 = int(n * p + np.sqrt(n * p * (1 - p)) * norm.ppf(q=alpha/2))
        ci = [-S_ord[k1], -S_ord[k2]]

        return k, k1, k2, x_hat, ci
    

    k_vals = {}
    k1_k2_vals = {}
    mc_est = {}
    ci_vals = {}
    for p in [0.05, 0.02, 0.01]: 
        k, k1, k2, x_hat, ci = plain(n, p)

        k_vals[p] = k
        k1_k2_vals[p] = (k2, k1)
        mc_est[p] = x_hat
        ci_vals[p] = f"[{ci[0]:.4f}, {ci[1]:.4f}]"
    
    df = pd.DataFrame(
        [k_vals, k1_k2_vals, mc_est, ci_vals],
        index=["k", "(k1, k2)", "Estimate x_hat", "95% C.I."]
    )

    df.columns.names=["p"]

    return df
    

In [9]:
results_4_4 = example_4_4()
results_4_4

p,0.0500,0.0200,0.0100
k,500,200,100
"(k1, k2)","(457, 542)","(172, 227)","(80, 119)"
Estimate x_hat,5.0622,6.8581,8.0504
95% C.I.,"[4.9155, 5.2116]","[6.6230, 7.1551]","[7.7394, 8.3728]"


**Example 4.5. Difficulty in Estimating Small Probabilities.** Consider the one-factor portfolio credit risk model in Example 1.9. Let $c_k$ denote the loss from the default of the $k$-th obligor. Then the total loss is
$$L = \sum_{k = 1}^{m} c_k \boldsymbol{1}_{\{X_k \geq x_k\}}.$$
Estimate the probability that $L$ exceeds a given large threshold $h$.

In [ ]:
def example_4_5():
    rng = np.random.default_rng(4198)

    # simulation parameters
    m = 3
    c = np.array([2, 1, 4])
    rho = np.array([0.2, 0.5, 0.8])
    x = np.array([1, 1, 2])

    def plain(n, h):
        Z = norm.rvs(0, 1, size=(n, 1), random_state=rng)
        eps = norm.rvs(0, 1, size=(n, m), random_state=rng)
        X = Z * rho + np.sqrt(1 - rho**2) * eps
        L = np.sum((X >= x) * c, axis=1)
        H = (L > h).astype(int)

        # compute mean est. and std. error
        v_hat = np.mean(H)
        se = sem(H)
        re = se / v_hat

        return v_hat, se, re

    mc_est = {}
    se_vals = {}
    re_vals = {}
    for n in [2_500, 10_000]:
        for h in [1, 2, 4]: 
            mean_est, std_err, rel_err = plain(n, h)

            key = (f"Sample size n={n}", f"{h}")
            mc_est[key] = mean_est
            se_vals[key] = std_err
            re_vals[key] = f"{rel_err * 100:.2f}%"
    
    df1 = pd.DataFrame(
        [mc_est, se_vals, re_vals],
        index=["M.C. Estimate", "S.E.", "R.E."]
    )

    # Ensure MultiIndex columns
    df1.columns = pd.MultiIndex.from_tuples(df1.columns, names=["", "Threshold h"])

    #
    mc_est = {}
    se_vals = {}
    re_vals = {}
    for n in [10_000, 40_000]:
        for h in [6, 8, 10]: 
            mean_est, std_err, rel_err = plain(n, h)

            key = (f"Sample size n={n}", f"{h}")
            mc_est[key] = mean_est
            se_vals[key] = std_err
            re_vals[key] =  "NaN" if np.isnan(rel_err) else f"{rel_err * 100:.2f}%"
    
    df2 = pd.DataFrame(
        [mc_est, se_vals, re_vals],
        index=["M.C. Estimate", "S.E.", "R.E."]
    )

    # Ensure MultiIndex columns
    df2.columns = pd.MultiIndex.from_tuples(df2.columns, names=["", "Threshold h"])

    return df1, df2

In [11]:
results_4_5 = example_4_5()
for result in results_4_5:
    display(result)

C:\Users\ningc\AppData\Local\Temp\ipykernel_5652\2271168701.py:20: RuntimeWarning: invalid value encountered in scalar divide
  re = se / v_hat


Sample size n=2500                Sample size n=10000         \
Threshold h                    1      2       4                   1      2   
M.C. Estimate             0.1760 0.0536  0.0148              0.1746 0.0501   
S.E.                      0.0076 0.0045  0.0024              0.0038 0.0022   
R.E.                       4.33%  8.41%  16.32%               2.17%  4.35%   

                      
Threshold h        4  
M.C. Estimate 0.0125  
S.E.          0.0011  
R.E.           8.89%

Sample size n=10000               Sample size n=40000         \
Threshold h                     6      8     10                   6      8   
M.C. Estimate              0.0033 0.0000 0.0000              0.0029 0.0000   
S.E.                       0.0006 0.0000 0.0000              0.0003 0.0000   
R.E.                       17.38%    NaN    NaN               9.31%    NaN   

                      
Threshold h       10  
M.C. Estimate 0.0000  
S.E.          0.0000  
R.E.             NaN